# 5: Time Series

The hand-in exercise for this topic is Task 3,4 and 6 from the notebook “TSA_Task”. This
means that you have to do the cleaning of dataset, then create features (at least 5 new
features should be created, and you should be able to justify why you created each of
the features). And then, you should train an XGBoost model on the dataset. Note that
you also need to do relevant train, test, validation split and be able to explain why you
chose a certain split. Lastly, you should calculate evaluation metrics: rmse and mae to
show performance of your model. The hyperparameter tuning part is not required.

---

### Imports:

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
import os
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

Get the dataset w kaggleHub

In [13]:
path = kagglehub.dataset_download("lakshmi25npathi/bike-sharing-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\danie\.cache\kagglehub\datasets\lakshmi25npathi\bike-sharing-dataset\versions\1


In [14]:
dataset_path = "C:/Users/danie/.cache/kagglehub/datasets/lakshmi25npathi/bike-sharing-dataset/versions/1"

files = os.listdir(dataset_path)
print(files)

['day.csv', 'hour.csv', 'Readme.txt']


In [15]:
file_path = "C:/Users/danie/.cache/kagglehub/datasets/lakshmi25npathi/bike-sharing-dataset/versions/1/hour.csv"

df = pd.read_csv(file_path)

---

### Task 3 from TSA_task:

* Clean and pre-process the dataset as required and prepare the data for modelling.

First look EDA:

Using .head(), .info() and .describe() function to get a initial understanding of the data we are handling.
Also checking if we need to adress any missing values or other inconsistencies.

In [16]:
# Look at first 10 records in the data frane
df.head(10)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
5,6,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1
6,7,2011-01-01,1,0,1,6,0,6,0,1,0.22,0.2727,0.80,0.0000,2,0,2
7,8,2011-01-01,1,0,1,7,0,6,0,1,0.20,0.2576,0.86,0.0000,1,2,3
8,9,2011-01-01,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0.0000,1,7,8
9,10,2011-01-01,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0.0000,8,6,14


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  17379 non-null  int64  
 16  cnt         17379 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


No missing values in this data, so no need to do cleaning/transformation for now

In [18]:
# Use this to ensure there are no absurd outliers before beginning analysis
df.describe()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,17379.0000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,8690.0000,2.501640,0.502561,6.537775,11.546752,0.028770,3.003683,0.682721,1.425283,0.496987,0.475775,0.627229,0.190098,35.676218,153.786869,189.463088
std,5017.0295,1.106918,0.500008,3.438776,6.914405,0.167165,2.005771,0.465431,0.639357,0.192556,0.171850,0.192930,0.122340,49.305030,151.357286,181.387599
min,1.0000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,4345.5000,2.000000,0.000000,4.000000,6.000000,0.000000,1.000000,0.000000,1.000000,0.340000,0.333300,0.480000,0.104500,4.000000,34.000000,40.000000
50%,8690.0000,3.000000,1.000000,7.000000,12.000000,0.000000,3.000000,1.000000,1.000000,0.500000,0.484800,0.630000,0.194000,17.000000,115.000000,142.000000
75%,13034.5000,3.000000,1.000000,10.000000,18.000000,0.000000,5.000000,1.000000,2.000000,0.660000,0.621200,0.780000,0.253700,48.000000,220.000000,281.000000
max,17379.0000,4.000000,1.000000,12.000000,23.000000,1.000000,6.000000,1.000000,4.000000,1.000000,1.000000,1.000000,0.850700,367.000000,886.000000,977.000000


There seem to be no extreme outliers in this data, so we proceed to analysis.

---

### Task 4 from TSA_task:

* Create the lag and rolling windows features for the "cnt" column such as: 1 day lag, 1 week lag, 1 month, etc. and last 3 day rolling

A 'lag' is a when a data record is shifted to a previous time step, it makes it possible to describe the history of records in the data, enabling time based analysis. Lets add the lag features of 1 day, 1 wekk and 1 month to the dataframe.

In [19]:
# Making lag features for the 'cnt' feature, trying to make the bike rental history easier to read
df['cnt_lag_1d'] = df['cnt'].shift(24)  # 1 day)
df['cnt_lag_1w'] = df['cnt'].shift(24 * 7)  # 1 week
df['cnt_lag_1m'] = df['cnt'].shift(24 * 30)  # 1 month

A rolling window feature calculates a mean sum over a specified window of time

In [20]:
# lets make a window of 3 days
df['cnt_roll_3d'] = df['cnt'].rolling(window=24 * 3, min_periods=1).mean()

Lets take a look at our new features to see if they seem useful

In [21]:
# Using data sample instead of data head here, as first few values will always be NaN since there are no previous data to calculate result from
df.sample(10)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,...,atemp,hum,windspeed,casual,registered,cnt,cnt_lag_1d,cnt_lag_1w,cnt_lag_1m,cnt_roll_3d
16484,16485,2012-11-24,4,1,11,14,0,6,0,1,...,0.2879,0.33,0.4627,58,149,207,435.0,555.0,27.0,136.888889
3524,3525,2011-05-31,2,0,5,18,0,2,1,1,...,0.8030,0.53,0.1940,55,362,417,249.0,564.0,159.0,178.291667
1981,1982,2011-03-28,2,0,3,10,0,1,1,1,...,0.2273,0.30,0.1940,22,47,69,54.0,18.0,189.0,91.791667
17053,17054,2012-12-18,4,1,12,8,0,2,1,1,...,0.3939,0.94,0.0000,10,652,662,592.0,724.0,42.0,196.861111
10078,10079,2012-03-01,1,1,3,0,0,4,1,2,...,0.4242,0.94,0.0000,1,10,11,65.0,62.0,181.0,146.250000
8118,8119,2011-12-09,4,0,12,22,0,5,1,1,...,0.3182,0.87,0.0000,15,114,129,92.0,102.0,161.0,105.638889
726,727,2011-02-02,1,0,2,15,0,3,1,1,...,0.3939,0.76,0.3284,10,66,76,52.0,61.0,2.0,57.541667
16084,16085,2012-11-07,4,1,11,21,0,3,1,2,...,0.3030,0.49,0.2985,5,121,126,151.0,192.0,86.0,222.388889
4513,4514,2011-07-11,3,0,7,23,0,1,1,1,...,0.6667,0.79,0.1343,11,44,55,72.0,138.0,131.0,198.652778
12443,12444,2012-06-07,2,1,6,16,0,4,1,1,...,0.6515,0.34,0.2239,77,389,466,452.0,487.0,389.0,296.541667


Makes it very conveniet to analyse variables like sales or rentals overtime

---

### Task 6 from TSA_task:

* Implement XGBoost to predict how many bikes will be rented each hour of the last week and evaluate using appropriate metrics.

To accomplish this task and train a XGBoost model, we must first make sure the data includes the following:

* Convert dteday to datetime format
* Create features 'hour', 'day', 'weekday', 'month' and 'year'
* Make a train/test split on the data

In [22]:
# Convert 'date' to datetime format
df['dteday'] = pd.to_datetime(df['dteday'])
# Create time-based features
df['hour'] = df['hr']
df['day'] = df['dteday'].dt.day
df['month'] = df['dteday'].dt.month
df['year'] = df['dteday'].dt.year
df['weekday'] = df['dteday'].dt.weekday  # 0 = Monday, 6 = Sunday
# Take a look at the dataframe after adding in the above
df.sample(10)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,...,registered,cnt,cnt_lag_1d,cnt_lag_1w,cnt_lag_1m,cnt_roll_3d,hour,day,month,year
883,884,2011-02-09,1,0,2,7,0,2,1,1,...,95,99,39.0,1.0,16.0,68.375000,7,9,2,2011
509,510,2011-01-23,1,0,1,8,0,6,0,1,...,17,19,8.0,83.0,NaN,62.097222,8,23,1,2011
2255,2256,2011-04-08,2,0,4,20,0,4,1,2,...,51,63,203.0,77.0,249.0,103.347222,20,8,4,2011
1240,1241,2011-02-24,1,0,2,19,0,3,1,3,...,112,115,209.0,88.0,50.0,78.097222,19,24,2,2011
12632,12633,2012-06-15,2,1,6,13,0,4,1,1,...,272,393,284.0,416.0,350.0,292.916667,13,15,6,2012
12492,12493,2012-06-09,2,1,6,17,0,5,0,1,...,257,495,793.0,586.0,732.0,322.097222,17,9,6,2012
5044,5045,2011-08-03,3,0,8,2,0,2,1,2,...,9,11,8.0,6.0,63.0,183.666667,2,3,8,2011
3177,3178,2011-05-17,2,0,5,7,0,1,1,2,...,138,151,272.0,314.0,16.0,164.847222,7,17,5,2011
12978,12979,2012-06-29,3,1,6,23,0,4,1,3,...,42,48,171.0,174.0,160.0,273.291667,23,29,6,2012
14976,14977,2012-09-21,3,1,9,5,0,4,1,1,...,47,49,56.0,41.0,40.0,270.847222,5,21,9,2012


These additions resulted in alot of columns in dataframe, to avoid confusing myself i will drop some unnecessary columns for this task.

In [23]:
drop_cols = ['instant', 'dteday', 'hr', 'casual', 'registered', 'yr', 'mnth']
df = df.drop(columns=drop_cols)
df.sample(10)

,season,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt,cnt_lag_1d,cnt_lag_1w,cnt_lag_1m,cnt_roll_3d,hour,day,month,year
11832,2,0,6,0,1,0.52,0.5000,0.72,0.1940,12,13.0,11.0,21.0,295.527778,5,13,5,2012
8301,4,0,5,0,2,0.28,0.2727,0.48,0.2537,236,137.0,286.0,121.0,148.055556,13,17,12,2011
10429,1,0,3,1,1,0.72,0.6515,0.37,0.3881,426,356.0,185.0,172.0,251.625000,16,15,3,2012
10128,1,0,5,0,3,0.36,0.3636,0.93,0.0896,38,6.0,46.0,140.0,140.055556,2,3,3,2012
13665,3,0,5,0,1,0.86,0.7727,0.39,0.2537,494,326.0,343.0,254.0,304.652778,14,28,7,2012
12418,2,0,2,1,1,0.64,0.6212,0.38,0.0000,300,272.0,275.0,262.0,285.611111,15,6,6,2012
3871,2,0,2,1,1,0.48,0.4697,0.67,0.2836,21,26.0,21.0,21.0,196.416667,5,15,6,2011
15852,4,0,5,0,2,0.56,0.5303,0.64,0.2836,618,817.0,731.0,905.0,327.986111,17,27,10,2012
3826,2,0,0,1,1,0.58,0.5455,0.49,0.4478,476,92.0,438.0,88.0,195.125000,8,13,6,2011
8984,1,0,6,0,2,0.16,0.1364,0.47,0.3284,5,4.0,6.0,72.0,138.263889,5,15,1,2012
